In [2]:
import numpy as np
import pandas as pd
from decimal import Decimal

Create the Hidden Markov Model Class

In [3]:
class HMM():
    def __init__(self):
        pass
    
    #initialize probability for forward propogation at first time interval
    def forward_initial(self, Observables, B):
        alpha_t=[0]*B.shape[0]
        index=Observables[0]
        for i in range(0,B.shape[0]):
            alpha_t[i]=PI[i]*B[i][index]
        return alpha_t
    
    # Calculate Forward Propogation Probabilities
    def forward_propagation(self,PI, A, B,Observables):
        forward_proba=np.zeros([PI.shape[0],Observables.shape[0]])
        alpha_t = self.forward_initial(Observables,B)
        forward_proba[:,0] = alpha_t
        for time in range(1,Observables.shape[0]):
            #compute an inner product with the first entry for inner summation
            index=Observables[time]
            temp=np.asarray(alpha_t)
            inner_product=[np.dot(temp,A[:,item])*B[item][index] for item in range(A.shape[1])]
            forward_proba[:,time]=np.asarray(inner_product)
            alpha_t=inner_product
        alpha = forward_proba # Rename
        return alpha
    
    # Calculate Normalized Forward Propogation Probabilities
    def forward_normalized(self,PI, A, B,Observables):
        forward_proba=np.zeros([PI.shape[0],len(Observables)])
        alpha_t = self.forward_initial(Observables,B)
        c=np.zeros(len(Observables))
        currScaleFact = 1/np.array(alpha_t).sum(axis=0)
        c[0] = currScaleFact
        alpha_t = [elem * currScaleFact for elem in alpha_t]
        forward_proba[:,0] = alpha_t
        for time in range(1,len(Observables)):
            #compute an inner product with the first entry for inner summation
            index=Observables[time]
            temp=np.asarray(alpha_t)
            inner_product=[np.dot(temp,A[:,item])*B[item][index] for item in range(A.shape[1])]
            # normalize this inner product
            currScaleFact = 1/np.array(inner_product).sum(axis=0)
            c[time] = currScaleFact
            inner_product = [elem * currScaleFact for elem in inner_product]
            forward_proba[:,time]=np.asarray(inner_product)
            alpha_t=inner_product
        alpha = forward_proba # Rename
        return alpha,c
    
    #Initialize probability for forward propogation at first time interval
    def backward_initial(self, Observables, B):
        beta_t=[0]*B.shape[0]
        index=Observables[-1]
        for i in range(0,B.shape[0]):
            beta_t[i]=1
        return beta_t
    
    # Calculate Backward Propogation Probabilities
    def backward_propagation(self,PI, A, B,Observables):
        backward_proba=np.empty([PI.shape[0],len(Observables)])
        beta_t = self.backward_initial(Observables,B)
        backward_proba[:,-1] = beta_t
        for time in reversed(range(0,len(Observables)-1)): # Goes over each time
            index=Observables[time+1] # index for current observation
            temp=np.asarray(beta_t) # Holds previous beta column
            inner_product=np.empty(PI.shape[0])
            for i in range(0,PI.shape[0]): # Go through each row to fill in new beta column
                temp_product = 0
                for j in range(0,PI.shape[0]): # Go through each row for summation from older beta column
                    temp_product+=B[j,index]*temp[j]*A[i,j]
                inner_product[i]=temp_product
            backward_proba[:,time]=np.asarray(inner_product)
            beta_t=inner_product
        beta = backward_proba # Rename
        return beta
    
    # Calculate Normalized Backward Propogation Probabilities
    def backward_normalized(self,PI, A, B,Observables,c):
        backward_proba=np.empty([PI.shape[0],len(Observables)])
        beta_t = self.backward_initial(Observables,B)
        # Normalize beta_t using c
        currScaleFact = c[len(Observables)-1]
        beta_t = [elem * currScaleFact for elem in beta_t]
        backward_proba[:,-1] = beta_t
        for time in reversed(range(0,len(Observables)-1)):
            #compute an inner product with the first entry
            index=Observables[time+1]
            temp=np.asarray(beta_t)
            inner_product=np.empty(PI.shape[0])
            for i in range(0,PI.shape[0]): # Go through each row to fill in new beta column
                temp_product = 0
                for j in range(0,PI.shape[0]): # Go through each row for summation from older beta column
                    temp_product+=B[j,index]*temp[j]*A[i,j]
                inner_product[i]=temp_product
            # Normalize inner_product
            currScaleFact = c[time]
            inner_product = [elem * currScaleFact for elem in inner_product]
            backward_proba[:,time]=np.asarray(inner_product)
            beta_t=inner_product
        beta = backward_proba # Rename
        return beta
    
    # Calculates probability for unnormalized forward observations occuring
    def evaluation_unnormalized(self,alpha):
        #grab last column of alpha
        eval_last=alpha[:,-1]
        eval_proba=eval_last.sum(axis=0)
        return eval_proba
    
    # Calculates log probabilities for normalized forward observations occuring
    def evaluation_normalized(self,c):
        apply_function=lambda t:np.log(t)
        c_log=np.vectorize(apply_function)
        eval_proba_norm=c_log(c)
        eval_proba_norm=-eval_proba_norm.sum(axis=0)
        return eval_proba_norm
    
    # Calculate smoothed probability for observations sequence occuring
    def smoothed_probability(self,alpha,beta):
        smoothed_proba=np.array([[alpha[i, j]*beta[i,j] for j in range(alpha.shape[1])]
                     for i in range(alpha.shape[0])])
        sum_sp=smoothed_proba.sum(axis=0)
        scale_function=lambda t:1/t
        c_func=np.vectorize(scale_function)
        c=c_func(sum_sp)
        sp_normalized=np.array([[smoothed_proba[i, j]*c[j] for j in range(alpha.shape[1])]
                     for i in range(alpha.shape[0])])
        smoothed_proba=sp_normalized
        return smoothed_proba

Create Initial Stage Probability Vector<br>
This holds the initial probabilities for each hidden state, here we considered only two.

In [4]:
# Our two hidden states are depressed and healthy
states = ['Depressed','Healthy']

# Below is the initial stage probabilities for each state
PI = np.array([.5,.5])

# Convert to pandas dataframe for better output display
PI_df = pd.DataFrame(PI,index=states)
print('The initial hidden state probability vector is below:\n')
print(PI_df.transpose())

The initial hidden state probability vector is below:

   Depressed  Healthy
0        0.5      0.5


Create Hidden State Transition Matrix<br>
This contains the probabilities of transitioning from one hidden state, given on side, to another hidden state, given along the top.

In [5]:
# equals transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states
#first  I make data frame then  I convert to matrix A
A_df = pd.DataFrame(columns=states, index=states)
A_df.loc[states[0]] = [0.999, 0.001]
A_df.loc[states[1]] = [0.2 ,0.8]
A_df.head()
A=np.asarray(A_df)

print('The hidden state transition matrix is below:\n')
print(A_df)

The hidden state transition matrix is below:

          Depressed Healthy
Depressed     0.999   0.001
Healthy         0.2     0.8


Create matrix of observation (emission) probabilities<br>
This contains the probability of the given emission (given on top) from a given hidden state (given on side)

In [6]:
# Below is the legend connecting observed actions with representative integers:
# Movement - 0
# Passive Social Networking - 1
# Active Social Networking - 2
# Texting - 3
# Access Psych Health App/Sites - 4

# These are the states which can be observed
states_obs=['0','1','2','3','4']

B_df = pd.DataFrame(columns=states_obs, index=states)
B_df.loc[states[0]] = [0.05, 0.35, 0.2,0.2,0.2]
B_df.loc[states[1]] = [0.5, 0.1, 0.3,0.1,0]
B=np.asarray(B_df)

print('The matrix of emission probabilities is below:\n')
print(B_df)

The matrix of emission probabilities is below:

              0     1    2    3    4
Depressed  0.05  0.35  0.2  0.2  0.2
Healthy     0.5   0.1  0.3  0.1    0


For testing the code below creates a vector containing a set of randomly observed actions of chosen length<br>
A more realistic example would come from actual data so there would be an actual pattern to uncover

In [15]:
len_obs = 10 # Length of observation sequence to create

# This is a randomly created dummy vector, which for real dataset would be replaced
Observables = np.random.randint(5, size=len_obs)
Observables=np.array(Observables)

# Convert to pandas dataframe for better output display
Observables_df=pd.DataFrame(Observables)
print('Below is the observation sequence used in this analysis (note row is marked with index 0 at far left)\n')
print(Observables_df.transpose().to_string(index=False,header=False))

Below is the observation sequence used in this analysis (note row is marked with index 0 at far left)

 0  2  4  4  4  1  4  4  2  1


Initialize the HMM class so for testing below is same as just calling a previously created definition

In [7]:
HMM_class = HMM()

Calculate the unnormalized forward variable matrix

In [8]:
alpha=HMM_class.forward_propagation(PI, A, B,Observables)
print('Below is the unnormalized forward variable matrix:\n')

# Convert to pandas dataframe for better output display
alpha_df=pd.DataFrame(alpha, index=[states])
print(alpha_df)

Below is the unnormalized forward variable matrix:

              0         1         2         3         4         5         6  \
Depressed  0.10  0.045465  0.009564  0.000507  0.000148  0.000041  0.000003   
Healthy    0.15  0.012010  0.002896  0.001163  0.000279  0.000067  0.000027   

                  7             8             9  
Depressed  0.000002  8.063373e-08  1.614291e-08  
Healthy    0.000000  8.071445e-10  2.179048e-10  


Calculate the normalized forward variable matrix and display normalization constants

In [9]:
[alpha_norm,c]=HMM_class.forward_normalized(PI, A, B,Observables)
print('Below is the normalized forward variable matrix:\n')

# Convert to pandas dataframe for better output display
alpha_norm_df=pd.DataFrame(alpha_norm, index=[states])
print(alpha_norm_df)
print('\n');print('\n');print('\n') # 3 empty spaces before displaying the scaling constants

print('Below are the scaling factors used in the normalization procedure (note row is marked with index 0 at far left):\n')
c_df=pd.DataFrame(c)
print(c_df.transpose())

Below is the normalized forward variable matrix:

             0        1        2         3         4         5         6    7  \
Depressed  0.4  0.79104  0.76758  0.303431  0.345987  0.377597  0.091473  1.0   
Healthy    0.6  0.20896  0.23242  0.696569  0.654013  0.622403  0.908527  0.0   

                  8         9  
Depressed  0.990089  0.986681  
Healthy    0.009911  0.013319  






Below are the scaling factors used in the normalization procedure (note row is marked with index 0 at far left):

     0         1         2         3         4         5         6          7  \
0  4.0  4.349717  4.612633  7.461752  3.909978  3.962662  3.646508  18.309208   

           8         9  
0  19.821606  4.977802  


Calculate the unnormalized backward variable matrix

In [10]:
beta = HMM_class.backward_propagation(PI,A,B,Observables)
print('Below is the unnormalized backward variable matrix:\n')

# Convert to pandas dataframe for better output display
beta_df=pd.DataFrame(beta, index=[states])
print(beta_df)

Below is the unnormalized backward variable matrix:

                      0             1             2         3         4  \
Depressed  1.527562e-08  4.334564e-08  2.095015e-07  0.000004  0.000020   
Healthy    9.888832e-08  1.198177e-06  4.957486e-06  0.000012  0.000048   

                  5         6         7       8    9  
Depressed  0.000101  0.002025  0.010135  0.2001  1.0  
Healthy    0.000182  0.000405  0.114001  0.2800  1.0  


Calculate the normalized backward variable matrix

In [11]:
beta_norm = HMM_class.backward_normalized(PI, A, B,Observables,c)
print('Below is the normalized backward variable matrix:\n')

# Convert to pandas dataframe for better output display
beta_norm_df=pd.DataFrame(beta_norm, index=[states])
print(beta_norm_df)

Below is the normalized backward variable matrix:

                  0          1          2         3         4         5  \
Depressed  0.933671   0.662339   0.735972  3.100600  2.072411  2.645660   
Healthy    6.044219  18.308638  17.415487  9.361502  4.882091  4.761655   

                   6           7          8         9  
Depressed  13.339580   18.309208  19.743473  4.977802  
Healthy     2.670587  205.946621  27.627049  4.977802  


Calculate indicated probabilities for observation sequence using the normalized forward variable matrix and the log probability for the normalized forward variable matrix

In [12]:
eval_proba=HMM_class.evaluation_unnormalized(alpha)
print("The unnormalized probability of the given observation sequence is: %.2E" %Decimal(eval_proba))
print('\n')

eval_proba_norm=HMM_class.evaluation_normalized(c)
print("The normalized log-probability of the given observation sequence is: %f" %eval_proba_norm)

The unnormalized probability of the given observation sequence is: 1.64E-08


The normalized log-probability of the given observation sequence is: -17.928377


Using Forward-Backward Smoothing this calculates the smoothed probability matrix

In [13]:
smoothed_proba=HMM_class.smoothed_probability(alpha,beta)
print('Below is the smoothed probability matrix:\n')

# Convert to pandas dataframe for better output display
smoothed_proba_df=pd.DataFrame(smoothed_proba, index=[states])
print(smoothed_proba_df)

Below is the smoothed probability matrix:

                  0         1         2         3         4         5  \
Depressed  0.093367  0.120453  0.122472  0.126085  0.183384  0.252102   
Healthy    0.906633  0.879547  0.877528  0.873915  0.816616  0.747898   

                  6    7         8         9  
Depressed  0.334623  1.0  0.986186  0.986681  
Healthy    0.665377  0.0  0.013814  0.013319  
